<a href="https://colab.research.google.com/github/KOMBOU12/Marius/blob/main/Adapt_feature_based_PRED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projet : ADAPTATION DE DOMAINE**
Le projet consiste à faire fonctionner les codes PRED, FA, CORAL de la toolbox https://adapt-python.github.io/adapt/ avec des données diverses de mon choix.

# **1- PRED**

PRED utilise la sortie d'un modèle préentraîné sur les données sources comme une caractéristique dans le modèle cible. Plus précisément, PRED entraîne d'abord un modèle SrcOnly. Ensuite, il exécute le modèle SrcOnly sur les données cibles. Les prédictions effectuées par le modèle SrcOnly sont utilisées comme caractéristiques supplémentaires, et un second modèle est entraîné sur les données cibles, enrichies avec cette nouvelle caractéristique.

In [1]:
!pip install adapt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for adapt: filename=adapt-0.4.4-py3-none-any.whl size=131790 sha256=0bf6d878d769ff180d3b855958b7eb785bd566989bb45a501e65cdea18ca8506
  Stored in directory: /root/.cache/pip/wheels/cb/88/91/f7ac7b6024ad8778b5ecca6ebe44421105722648f2a6b7afd7
Successfully built adapt


On reproduit le code sur un nouveau jeu de données **breast cancer**

In [2]:
from sklearn.linear_model import RidgeClassifier
from sklearn.datasets import load_breast_cancer
from adapt.feature_based import PRED
from sklearn.model_selection import train_test_split

# Charger le jeu de données sur le cancer du sein
data = load_breast_cancer()
X, y = data.data, data.target

# Séparer les données en ensemble source (Xs, ys) et ensemble cible (Xt, yt)
Xs, Xt, ys, yt = train_test_split(X, y, test_size=0.5, random_state=0)

# Initialiser et entraîner le modèle PRED avec RidgeClassifier
model = PRED(RidgeClassifier(alpha=1.0), Xt=Xt[:3], yt=yt[:3],
             pretrain=True, verbose=0, random_state=0)
model.fit(Xs, ys)

# Évaluer le modèle
score = model.score(Xt, yt)
print("Score:", score)


Score: 0.9333333333333333


Ce score est encourageant et montre que le modèle est performant dans cette tâche spécifique, mais une analyse plus approfondie avec des métriques complémentaires et des tests sur d'autres données permettrait de confirmer la validité et la robustesse de cette performance.

# **2- FA**

FA consiste en une méthode d'augmentation des caractéristiques où chaque vecteur de caractéristiques d'entrée est transformé de la manière suivante :

1.   Les vecteurs de caractéristiques d'entrée de la source  $X_s$ sont transformés en $(X_s, 0, X_s)$
2.   Les vecteurs de caractéristiques d'entrée de la cible $ X_t $ sont transformés en $ (0, X_t, X_t)$.

Où $0$ désigne le vecteur nul de même dimension que  $X_s$ et $X_t$.



On reproduit le code sur le jeu de données **digits**

In [3]:
from sklearn.linear_model import RidgeClassifier
from sklearn.datasets import load_digits
from adapt.feature_based import FA
from sklearn.model_selection import train_test_split

# Charger le jeu de données digits
data = load_digits()
X, y = data.data, data.target

# Diviser les données en ensembles source (Xs, ys) et cible (Xt, yt)
Xs, Xt, ys, yt = train_test_split(X, y, test_size=0.5, random_state=0)

# Initialiser et entraîner le modèle FA avec RidgeClassifier
# En utilisant un sous-ensemble de Xt et yt pour l'adaptation de domaine
model = FA(RidgeClassifier(), Xt=Xt[:10], yt=yt[:10], random_state=0)
model.fit(Xs, ys)

# Évaluer le modèle
score = model.score(Xt, yt)
print("Score:", score)


Fit transform...
Previous shape: (898, 64)
New shape: (908, 192)
Fit Estimator...
Score: 0.6162402669632926




*   La transformation a enrichi les données. La dimension initiale des caractéristiques (64) est passée à 192.
*   Le score de 61,6 % représente l'exactitude sur les données cibles.
*   Compte tenu que seulement 10 exemples du domaine cible ont été utilisés pour guider l'adaptation, cette performance est raisonnable. Cependant, il reste de la marge pour améliorer l'alignement entre les distributions source et cible.

# **3- CORAL**

CORAL est une méthode d'adaptation de domaine basée sur les caractéristiques, qui minimise le décalage entre domaines en alignant les statistiques d'ordre deux des distributions source et cible.

La méthode transforme les caractéristiques de la source afin de minimiser la norme de Frobenius entre la matrice de corrélation des données cibles d'entrée et celle des données sources transformées.

On reproduit le code sur le jeu de données **wine**

In [ ]:
from sklearn.linear_model import RidgeClassifier
from sklearn.datasets import load_wine
from adapt.feature_based import CORAL
from sklearn.model_selection import train_test_split

# Charger le jeu de données wine
data = load_wine()
X, y = data.data, data.target

# Diviser les données en ensembles source (Xs, ys) et cible (Xt, yt)
Xs, Xt, ys, yt = train_test_split(X, y, test_size=0.5, random_state=0)

# Initialiser et entraîner le modèle CORAL avec RidgeClassifier
model = CORAL(RidgeClassifier(), Xt=Xt, random_state=0)
model.fit(Xs, ys)

# Évaluer le modèle
score = model.score(Xt, yt)
print("Score:", score)


Fit transform...
Previous covariance difference: 35.215205
New covariance difference: 0.000002
Fit Estimator...
Score: 0.8426966292134831
